In [1]:
# ============================
# United ORD EDA - Full Script (FIXED)
# ============================

# 0) Imports & config
import os
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["axes.grid"] = True

import statsmodels.api as sm
from scipy import stats

# ----------------------------
# Config
# ----------------------------
INPUT_CSV = "/content/final_flight_data.csv"   # <<<< UPDATE THIS PATH
OUTPUT_DIR = Path("new_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# Column mapping from your CSV -> canonical names used below
COLS = {
    "company_id": "company_id",
    "flight_number": "flight_number",
    "flight_date": "flight_date",   # date key you exported (scheduled_departure_date_local)
    "orig": "scheduled_departure_station_code",
    "dest": "scheduled_arrival_station_code",
    "sched_dep_dt": "scheduled_departure_datetime_local",
    "act_dep_dt": "actual_departure_datetime_local",
    "sched_arr_dt": "scheduled_arrival_datetime_local",
    "act_arr_dt": "actual_arrival_datetime_local",

    "scheduled_ground_time_minutes": "scheduled_ground_time_minutes",
    "actual_ground_time_minutes": "actual_ground_time_minutes",
    "minimum_turn_minutes": "minimum_turn_minutes",

    "total_seats": "total_seats",
    "fleet_type": "fleet_type",
    "carrier": "carrier",

    # Aggregated inputs from SQL
    "pax_total": "pax_total",
    "pax_basic_econ": "pax_basic_econ",
    "lap_childs": "lap_childs",
    "pax_childs": "pax_childs",
    "stroller_users": "stroller_users",

    "ssr_count": "ssr_count",
    "ssr_wchr": "ssr_wchr",

    "checked_bags": "checked_bags",
    "transfer_bags": "transfer_bags",
    "hot_transfer_bags": "hot_transfer_bags",
}

DATE_COLS = ["sched_dep_dt", "act_dep_dt", "sched_arr_dt", "act_arr_dt"]

# ----------------------------
# 1) Load data
# ----------------------------
df = pd.read_csv(INPUT_CSV, low_memory=False)

# Standardize expected columns by renaming first
df = df.rename(columns={v:k for k,v in COLS.items()})  # reverse-map to canonical names used below

# Convert date columns to datetime objects after renaming
for col in DATE_COLS:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Derive flight_date from the (renamed) scheduled departure, if not present
if "flight_date" not in df.columns or df["flight_date"].isna().all():
    if "sched_dep_dt" in df.columns:
        df["flight_date"] = pd.to_datetime(df["sched_dep_dt"], errors='coerce').dt.date
else:
    df["flight_date"] = pd.to_datetime(df["flight_date"], errors='coerce').dt.date

# ----------------------------
# 2) Basic QC
# ----------------------------
df = df.copy()

# Coerce numeric cols
num_cols = [
    "scheduled_ground_time_minutes","actual_ground_time_minutes","minimum_turn_minutes",
    "total_seats","pax_total","pax_basic_econ","lap_childs","pax_childs","stroller_users",
    "ssr_count","ssr_wchr","checked_bags","transfer_bags","hot_transfer_bags"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ----------------------------
# 3) Feature engineering
# ----------------------------
# 3.1 Delay minutes (floor at 0)
dep_delay_min = (df["act_dep_dt"] - df["sched_dep_dt"]).dt.total_seconds().div(60)
df["dep_delay_min"] = dep_delay_min.clip(lower=0)
df["dep_delay_flag"] = (df["dep_delay_min"] > 0).astype("Int64")

# 3.2 Turn pressure
df["turn_buffer"] = df["scheduled_ground_time_minutes"] - df["minimum_turn_minutes"]
df["near_min_turn_flag_5"]  = (df["turn_buffer"] <= 5).astype("Int64")
df["near_min_turn_flag_0"]  = (df["turn_buffer"] <= 0).astype("Int64")
df["near_min_turn_flag_10"] = (df["turn_buffer"] <= 10).astype("Int64")

# 3.3 Bags & safer ratios
# If the bag feed is missing for a flight, leave all three as NaN (not zero)
has_any_bag = df["checked_bags"].notna() | df["transfer_bags"].notna() | df["hot_transfer_bags"].notna()
df.loc[~has_any_bag, ["checked_bags","transfer_bags","hot_transfer_bags"]] = np.nan

# Fill remaining NAs with 0 only where feed exists
df["checked_bags"] = df["checked_bags"].fillna(0)
df["transfer_bags"] = df["transfer_bags"].fillna(0)
df["hot_transfer_bags"] = df["hot_transfer_bags"].fillna(0)

# Ratios only when denominators valid
mask_ratio = df["checked_bags"].gt(0)
df["transfer_to_checked_ratio"] = np.where(mask_ratio,
                                           df["transfer_bags"] / df["checked_bags"],
                                           np.nan)
den = (df["transfer_bags"] + df["checked_bags"])
df["transfer_share"] = np.where(den.gt(0), df["transfer_bags"]/den, np.nan)

# 3.4 Loads & SSR
# If lap infants inflate pax_total and you want seated pax only:
if "lap_childs" in df.columns and df["lap_childs"].notna().any():
    df["pax_adj"] = (df["pax_total"] - df["lap_childs"]).clip(lower=0)
else:
    df["pax_adj"] = df["pax_total"]

df["total_seats"] = df["total_seats"].replace({0: np.nan})
df["load_factor_proxy"] = df["pax_adj"] / df["total_seats"]
# Cap to [0,1] for EDA sanity
df["load_factor_proxy"] = df["load_factor_proxy"].clip(lower=0, upper=1)

df["ssr_count"] = df["ssr_count"].fillna(0)
df["ssr_wchr"]  = df["ssr_wchr"].fillna(0)
df["ssr_rate"]  = df["ssr_count"] / df["pax_adj"].replace({0: np.nan})
df["wchr_rate"] = df["ssr_wchr"]  / df["pax_adj"].replace({0: np.nan})

# 3.5 Time-of-day
df["hour_of_day"] = df["sched_dep_dt"].dt.hour

# (Optional) Composite difficulty label (not required for EDA)
p85 = df.groupby("flight_date", observed=False)["pax_adj"].transform(lambda s: np.nanpercentile(s.dropna(), 85) if s.notna().any() else np.nan)
df["turn_buffer_shortfall"] = df["minimum_turn_minutes"] - df["actual_ground_time_minutes"]
df["difficulty_label"] = (
    (df["dep_delay_min"] >= 15) |
    (df["turn_buffer_shortfall"] >= 0) |
    (df["hot_transfer_bags"] >= 5) |
    ((df["ssr_wchr"] >= 5) & (df["pax_adj"] >= p85))
).astype("Int64")

# ----------------------------
# 4) EDA questions & outputs
# ----------------------------
def save_table(df_tbl, name):
    path = OUTPUT_DIR / f"{name}.csv"
    df_tbl.to_csv(path, index=False)
    print(f"Saved: {path.resolve()}")

# Q1
eda_delay = pd.DataFrame({
    "avg_delay_min": [df["dep_delay_min"].mean(skipna=True)],
    "median_delay_min": [df["dep_delay_min"].median(skipna=True)],
    "pct_late_departures": [df["dep_delay_flag"].mean(skipna=True) * 100]
})
print("\n[Q1] Average delay & % late:"); print(eda_delay.round(2))
save_table(eda_delay.round(2), "q1_avg_delay_and_pct_late")

by_slice = (df
    .groupby(["fleet_type","carrier"], dropna=False, observed=False)
    .agg(avg_delay_min=("dep_delay_min","mean"),
         pct_late=("dep_delay_flag","mean"),
         n=("flight_number","count"))
    .reset_index())
by_slice["pct_late"] = by_slice["pct_late"]*100
save_table(by_slice.round(2), "q1_by_fleet_and_carrier")

# Q2
turn_summary = pd.DataFrame({
    "count_turn_buffer_le_0": [int(df["near_min_turn_flag_0"].sum(skipna=True))],
    "count_turn_buffer_le_5": [int(df["near_min_turn_flag_5"].sum(skipna=True))],
    "count_turn_buffer_le_10": [int(df["near_min_turn_flag_10"].sum(skipna=True))],
    "total_flights": [len(df)]
})
turn_summary["pct_le_0"]  = 100 * turn_summary["count_turn_buffer_le_0"]  / turn_summary["total_flights"]
turn_summary["pct_le_5"]  = 100 * turn_summary["count_turn_buffer_le_5"]  / turn_summary["total_flights"]
turn_summary["pct_le_10"] = 100 * turn_summary["count_turn_buffer_le_10"] / turn_summary["total_flights"]
print("\n[Q2] Turn buffer (scheduled vs minimum):"); print(turn_summary.round(2))
save_table(turn_summary.round(2), "q2_turn_buffers")

# Q3
bag_ratio_summary = pd.DataFrame({
    "mean_transfer_to_checked_ratio": [df["transfer_to_checked_ratio"].mean(skipna=True)],
    "median_transfer_to_checked_ratio": [df["transfer_to_checked_ratio"].median(skipna=True)],
    "mean_transfer_share": [df["transfer_share"].mean(skipna=True)],
    "median_transfer_share": [df["transfer_share"].median(skipna=True)],
})
print("\n[Q3] Transfer vs checked ratios:"); print(bag_ratio_summary.round(3))
save_table(bag_ratio_summary.round(3), "q3_bag_ratios")

# Q4
valid = df[["load_factor_proxy","dep_delay_min"]].dropna()
if len(valid) >= 2:
    r, p = stats.pearsonr(valid["load_factor_proxy"], valid["dep_delay_min"])
else:
    r, p = (np.nan, np.nan)

loads_summary = pd.DataFrame({
    "mean_load_factor": [df["load_factor_proxy"].mean(skipna=True)],
    "median_load_factor": [df["load_factor_proxy"].median(skipna=True)],
    "pearson_r_load_vs_delay": [r],
    "pearson_p_value": [p]
})
print("\n[Q4] Loads vs delay (correlation):"); print(loads_summary.round(3))
save_table(loads_summary.round(3), "q4_loads_vs_delay")

# LF distribution
try:
    ax = df["load_factor_proxy"].dropna().plot(kind="hist", bins=20, alpha=0.7, edgecolor="black")
    ax.set_title("Distribution of Load Factor (proxy)")
    ax.set_xlabel("Load factor (pax_adj / seats)")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "q4_load_factor_distribution.png", dpi=150)
    plt.close()
except Exception as e:
    print(f"(Chart skipped) {e}")

# Q5 — GLM (Poisson) with robust SE
model_df = df[
    ["dep_delay_min","wchr_rate","ssr_rate","load_factor_proxy","turn_buffer",
     "hot_transfer_bags","hour_of_day","fleet_type","carrier"]
].copy()

# Clean: remove inf, enforce required fields, dummies, numeric types
model_df = model_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[
    "dep_delay_min","wchr_rate","ssr_rate","load_factor_proxy",
    "turn_buffer","hot_transfer_bags","hour_of_day","fleet_type","carrier"
])
model_df = pd.get_dummies(model_df, columns=["fleet_type","carrier"], drop_first=True)
for c in model_df.columns:
    model_df[c] = pd.to_numeric(model_df[c], errors="coerce")
model_df = model_df.dropna()

X = sm.add_constant(model_df.drop(columns=["dep_delay_min"]).astype(float))
y = model_df["dep_delay_min"].astype(float)

try:
    glm = sm.GLM(y, X, family=sm.families.Poisson())
    glm_res = glm.fit(cov_type="HC3")
    print("\n[Q5] GLM Poisson: dep_delay_min ~ wchr_rate + controls")
    print(glm_res.summary())

    # Coefficients + IRR
    coef = glm_res.params.rename("coef").to_frame()
    coef["std_err"] = glm_res.bse
    coef["z"] = coef["coef"] / coef["std_err"]
    coef["p_value"] = glm_res.pvalues
    coef["IRR"] = np.exp(coef["coef"])
    coef["IRR_minus_1_pct"] = (coef["IRR"] - 1.0) * 100
    coef = coef.reset_index().rename(columns={"index":"feature"})
    coef = coef.sort_values("feature")
    coef.to_csv(OUTPUT_DIR / "q5_glm_coefficients_with_IRR.csv", index=False)
    print("Saved:", (OUTPUT_DIR / "q5_glm_coefficients_with_IRR.csv").resolve())

    # Key effects subset
    key_effects = coef[coef["feature"].isin(["wchr_rate","ssr_rate","load_factor_proxy","turn_buffer","hot_transfer_bags"])]
    key_effects.to_csv(OUTPUT_DIR / "q5_key_effects.csv", index=False)
    print("Saved:", (OUTPUT_DIR / "q5_key_effects.csv").resolve())

    # Over-dispersion quick check
    mean_y = y.mean(); var_y = y.var()
    with open(OUTPUT_DIR / "q5_overdispersion.txt","w") as fh:
        fh.write(f"Mean(dep_delay_min)={mean_y:.3f}, Var(dep_delay_min)={var_y:.3f}\n")
        fh.write("If Var >> Mean, consider Negative Binomial as robustness.\n")
    print("Saved:", (OUTPUT_DIR / "q5_overdispersion.txt").resolve())

    # Optional: Negative Binomial sensitivity
    try:
        nb = sm.GLM(y, X, family=sm.families.NegativeBinomial(alpha=1.0))
        nb_res = nb.fit(cov_type="HC3")
        nb_coef = nb_res.params.rename("coef").to_frame()
        nb_coef["IRR"] = np.exp(nb_coef["coef"])
        nb_coef = nb_coef.reset_index().rename(columns={"index":"feature"})
        nb_coef.to_csv(OUTPUT_DIR / "q5_negative_binomial_coefficients.csv", index=False)
        print("Saved:", (OUTPUT_DIR / "q5_negative_binomial_coefficients.csv").resolve())
    except Exception as e:
        print(f"NB fit skipped: {e}")

except Exception as e:
    print(f"GLM failed: {e}")

# ----------------------------
# 5) Helpful cross-tabs & visuals
# ----------------------------
# Delay by hour of day
by_hour = (df
    .groupby("hour_of_day", dropna=False, observed=False)
    .agg(avg_delay_min=("dep_delay_min","mean"),
         pct_late=("dep_delay_flag","mean"),
         n=("flight_number","count"))
    .reset_index())
by_hour["pct_late"] = by_hour["pct_late"]*100
save_table(by_hour.round(2), "xtab_delay_by_hour")

try:
    ax = by_hour.plot(x="hour_of_day", y="avg_delay_min", kind="line", marker="o")
    ax.set_title("Average Departure Delay by Scheduled Hour")
    ax.set_xlabel("Hour of day (local)")
    ax.set_ylabel("Avg delay (min)")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "xtab_delay_by_hour.png", dpi=150)
    plt.close()
except Exception as e:
    print(f"(Chart skipped) {e}")

# Delay vs. turn buffer buckets
by_turn = (df
    .assign(turn_bucket=pd.cut(df["turn_buffer"], bins=[-999,0,5,10,20,999],
                               labels=["<=0","(0,5]","(5,10]","(10,20]",">20"]))
    .groupby("turn_bucket", dropna=False, observed=False)
    .agg(avg_delay_min=("dep_delay_min","mean"),
         pct_late=("dep_delay_flag","mean"),
         n=("flight_number","count"))
    .reset_index())
by_turn["pct_late"] = by_turn["pct_late"]*100
save_table(by_turn.round(2), "xtab_delay_by_turn_buffer")

# ----------------------------
# 6) Final sanity printouts
# ----------------------------
print("\nEDA complete. Key outputs written to:", OUTPUT_DIR.resolve())

print("\n== Summary Answers ==")
print(f"- Average dep delay (min): {df['dep_delay_min'].mean(skipna=True):.2f}")
print(f"- % flights late: {100*df['dep_delay_flag'].mean(skipna=True):.2f}%")
print(f"- Flights with scheduled ground time <= minimum: {int(df['near_min_turn_flag_0'].sum(skipna=True))} "
      f"({100*df['near_min_turn_flag_0'].mean(skipna=True):.2f}%)")
print(f"- Mean transfer/checked ratio (valid cases only): {df['transfer_to_checked_ratio'].mean(skipna=True):.3f}")
print(f"- Correlation(load factor, delay): r={r:.3f}, p={p:.3g}")
print("  (See q5_glm_coefficients_with_IRR.csv for IRRs on wchr_rate, turn_buffer, hot_transfer_bags, etc.)")



[Q1] Average delay & % late:
   avg_delay_min  median_delay_min  pct_late_departures
0          23.35               0.0                49.61
Saved: /content/new_outputs/q1_avg_delay_and_pct_late.csv
Saved: /content/new_outputs/q1_by_fleet_and_carrier.csv

[Q2] Turn buffer (scheduled vs minimum):
   count_turn_buffer_le_0  count_turn_buffer_le_5  count_turn_buffer_le_10  \
0                     652                     780                     1183   

   total_flights  pct_le_0  pct_le_5  pct_le_10  
0           8099      8.05      9.63      14.61  
Saved: /content/new_outputs/q2_turn_buffers.csv

[Q3] Transfer vs checked ratios:
   mean_transfer_to_checked_ratio  median_transfer_to_checked_ratio  \
0                             NaN                               NaN   

   mean_transfer_share  median_transfer_share  
0                  1.0                    1.0  
Saved: /content/new_outputs/q3_bag_ratios.csv

[Q4] Loads vs delay (correlation):
   mean_load_factor  median_load_factor  p

In [5]:
import shutil
import os

output_dir = "new_outputs"
zip_file_name = "new_outputs.zip"

# Create a zip archive of the output directory
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', output_dir)

# Provide a link to download the zip file
from google.colab import files
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>